# Bert-VITS2 2.3 전용 추론 코랩
다른 VITS 계열 추론 안 됨

다른 버전(2.2, 2.1등) 추론 안 해봄

G.pth과 json 파일이 필수입니다

## **※※※CPU에서도 가능※※※**


> 코랩은 정책상 WebUI를 이용하는 것을 금지하고 있습니다. 이 코랩을 이용하는 모든 이용자는, 이 코랩을 이용하는 것으로 인해 **코랩 영구 사용 불가** 등의 불이익을 당할 수도 있다는 사실을 인지하였다고 간주합니다.



In [ ]:
# @title 준비
!git clone https://github.com/fishaudio/Bert-VITS2.git
#!git checkout 87462fe50f3f79ebf77600bac89548f192352d0c
#!python -m venv venv
#!venv\Scripts\activate

%cd Bert-VITS2
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
#!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117
!pip install -r requirements.txt
#!pip install --upgrade pip setuptools numpy numba
#!pip uninstall tensorflow
#!pip install tensorflow[and-cuda] --upgrade --force-reinstall

import os.path
from os import path

Cloning into 'Bert-VITS2'...
remote: Enumerating objects: 2396, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 2396 (delta 114), reused 148 (delta 77), pack-reused 2172
Receiving objects: 100% (2396/2396), 9.61 MiB | 8.72 MiB/s, done.
Resolving deltas: 100% (1402/1402), done.
/content/Bert-VITS2
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.5/751.5 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# @title 드라이브 마운트
#@markdown 자신의 드라이브에서 모델을 다운로드할 경우 런타임당 1회 실행합니다. ('/content/drive/...')

#@markdown 자신의 드라이브라고 해도 공유 링크를 사용하여 모델을 다운로드할 경우에는 필요 없습니다. ('drive.google.com...')
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title 모델 로딩

#@markdown ###한 번에 여러 모델을 다운받아 사용할 수 없습니다.
#@markdown 한 런타임 당 하나의 모델만 사용해주세요

#@markdown Google Drive 링크 또는 Hugging Face 링크에서의 다운로드만을 지원합니다.

#@markdown WebUI를 껐다가 다른 모델을 불러오기 위해서는 아래 clean_model_folder를 체크하여 이전 모델을 삭제하여주세요.

clean_model_folder = True # @param {type:"boolean"}


import os
import shutil
import urllib.request
import gdown
from urllib.parse import urlparse, parse_qs
import urllib.parse
from google.oauth2.service_account import Credentials
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests
from requests import get
import hashlib



inference_arrive = '/content/Bert-VITS2/Data/models/'
inference_json_arrive = '/content/Bert-VITS2/Data/'
os.makedirs(inference_json_arrive, exist_ok=True)
dir_name = inference_arrive.split('/')[-1]

G_PTH_WHERE = "/content/drive/MyDrive/Bert-VITS2_Khezu/Miyu/models/G_6000.pth" # @param {type:"string"}
SPEAKER_JSON_WHERE = "/content/drive/MyDrive/Bert-VITS2_Khezu/Miyu/config.json" # @param {type:"string"}

if clean_model_folder is True:
  if os.path.exists(inference_arrive):
    shutil.rmtree(inference_arrive)
    print(f'[-] 폴더를 청소하였습니다! 새로운 모델을 다운받아서 사용할 수 있습니다.')

os.makedirs(inference_arrive, exist_ok=True)

#%cd inference_folder

def download_model(url, dir_name):
    try:
        #extraction_folder = os.path.join(rvc_models_dir, dir_name)

        download_name = url.split('/')[-1]
        if '?download=true' in download_name:
          download_name = download_name.split('?')[-2]
        #download_where = '/inference_folder/' +  url.split('/')[-1]
        download_where = inference_arrive +  download_name

        print(f'[~] {download_name}를 허깅페이스에서 다운받고 있어요...')


        #urllib.request.urlretrieve(url, download_where)
        #urllib.request.urlretrieve(url, '/content/VITS-fast-fine-tuning/inference_folder')

        #print('[~] Extracting zip...')
        #extract_zip(extraction_folder, download_where)
        if "/blob/" in url:
          url = url.replace("blob", "resolve")
          print("Resolved URL:", url)  # Print the resolved URL
          !wget "$url" -O "$download_where"
          #/content/VITS-fast-fine-tuning/inference_folder
        else:
          !wget "$url" -O "$download_where"
          #/content/VITS-fast-fine-tuning/inference_folder
          #urllib.request.urlretrieve(url, '/content/VITS-fast-fine-tuning/inference_folder/')
        print(f'[+] {download_name}를 허깅페이스에서 성공적으로 다운로드 했어요!')

    except Exception as e:
        raise Exception(str(e))

def download_online_model_drive(file_id, dir_name):
    try:
        download_where = inference_arrive + '/'

        print(f'[~] 링크 공유된 구글드라이브에서 파일을 다운받고 있어요...')

        gdown.download(f'https://drive.google.com/uc?id={file_id}', download_where, quiet=False)

        #downloaded_where = max(glob(os.path.join(download_where, "*")), key=os.path.getctime)
        #download_where = downloaded_where.split('/')[-1]
        #print(f'[+] {download_name}를 링크 공유된 구글드라이브에서 성공적으로 다운로드 했어요!')
        print(f'[+] 링크 공유된 구글드라이브에서 파일을 성공적으로 다운로드 했어요!')

    except Exception as e:
        raise Exception(str(e))
def download_from_your_drive(url, dir_name):
    try:
        download_name = url.split('/')[-1]
        download_where = inference_arrive + '/' +  url.split('/')[-1]

        print(f'[~] {download_name}를 마운트 된 구글드라이브에서 다운받고 있어요...')

        !cp {url} {download_where}

        print(f'[+] {download_name}를 마운트 된 구글드라이브에서 성공적으로 다운로드 했어요!')
    except Exception as e:
        raise Exception(str(e))

# Detecta si la URL es de Google Drive o Hugging Face
if 'drive.google.com' in G_PTH_WHERE:
    file_id_drive = G_PTH_WHERE.split('/')[-2]
    download_online_model_drive(file_id_drive, dir_name)
elif 'huggingface.co' in G_PTH_WHERE:
    download_model(G_PTH_WHERE, dir_name)
elif '/content/drive' in G_PTH_WHERE:
  download_from_your_drive(G_PTH_WHERE, dir_name)
else:
    print('모르는 URL이네요... G_PTH_WHERE가 Google Drive 링크 또는 Hugging Face 링크에서만 다운받을 수 있어요!')

if 'drive.google.com' in SPEAKER_JSON_WHERE:
    file_id_drive = SPEAKER_JSON_WHERE.split('/')[-2]
    download_online_model_drive(file_id_drive, dir_name)
elif 'huggingface.co' in SPEAKER_JSON_WHERE:
    download_model(SPEAKER_JSON_WHERE, dir_name)
elif '/content/drive' in SPEAKER_JSON_WHERE:
  download_from_your_drive(SPEAKER_JSON_WHERE, dir_name)
else:
    print('모르는 URL이네요... SPEAKER_JSON_WHERE가 Google Drive 링크 또는 Hugging Face 링크에서만 다운받을 수 있어요!')


#%cd ..


[-] 폴더를 청소하였습니다! 새로운 모델을 다운받아서 사용할 수 있습니다.
[~] G_6000.pth를 마운트 된 구글드라이브에서 다운받고 있어요...
[+] G_6000.pth를 마운트 된 구글드라이브에서 성공적으로 다운로드 했어요!
[~] config.json를 마운트 된 구글드라이브에서 다운받고 있어요...
[+] config.json를 마운트 된 구글드라이브에서 성공적으로 다운로드 했어요!


In [ ]:
# @title 다운로드2
!python preprocess_text.py
!python preprocess_text.py
#2.1
!curl -L -o /content/Bert-VITS2/emotional/wav2vec2-large-robust-12-ft-emotion-msp-dim/pytorch_model.bin https://huggingface.co/audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim/resolve/main/pytorch_model.bin
#2.2
!curl -L -o /content/Bert-VITS2/emotional/clap-htsat-fused/pytorch_model.bin https://huggingface.co/laion/clap-htsat-fused/resolve/main/pytorch_model.bin
#2.3
!curl -L -o slm/wavlm-base-plus/pytorch_model.bin https://huggingface.co/microsoft/wavlm-base-plus/resolve/main/pytorch_model.bin


已根据默认配置文件default_config.yml生成配置文件config.yml。请按该配置文件的说明进行配置后重新运行。
如无特殊需求，请勿修改default_config.yml或备份该文件。
pytorch_model.bin: 100% 1.32G/1.32G [00:09<00:00, 139MB/s]
pytorch_model.bin: 100% 1.31G/1.31G [00:09<00:00, 138MB/s]
spm.model: 100% 2.46M/2.46M [00:00<00:00, 161MB/s]
pytorch_model.bin: 100% 874M/874M [00:08<00:00, 100MB/s]
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.
Usage: preprocess_text.py [OPTIONS]
Try 'preprocess_text.py --help' for help.

Error: Invalid value for '--transcription-path': File 'Data/filelists/你的数据

In [ ]:
# @title yaml 변경 (아마 실행할 필요 없음)

import yaml
with open('/content/Bert-VITS2/config.yml') as f:
  config = yaml.load(f, Loader=yaml.FullLoader)
#config['dataset_path'] = "Data/" + model_name + '/'
config['dataset_path'] = "Data/"
config['preprocess_text']['transcription_path'] = 'filelists/text.list'
#config['train_ms']['keep_ckpts'] = -1 #모든 모델 저장
#config['server']['models']['language'] = 'JP'
'''
config['dataset_path'] = "" #특정 경로 지정시 모든 경로는 상대적으로 입력되어야함. 아닐 경우 루트 기준.
#yml의 대부분이 상대경로 기준이므로, 로컬 학습시는 dataset_path를 "Data/model_name/"로 바꾸고 일부만 바꿔서 사용하는 게 편함
#난 상대경로 쓰는 법 몰라!
config['preprocess_text']['transcription_path'] = txt_file_arrive_where
resample:in_dir:'/Data/'+model_name+'/'+"audios/raw"
resample:out_dir:'/Data/'+model_name+'/'+"audios/wavs"
preprocess_text:transcription_path:'/Data/'+model_name+'/'+"filelists/text.list"
preprocess_text:train_path:'/Data/'+model_name+'/'+"filelists/train.list
preprocess_text:val_path:'/Data/'+model_name+'/'+"filelists/val.list
preprocess_text:config_path:'/Data/'+model_name+'/'+"config.json"
bert_gen:config_path:'/Data/'+model_name+'/'+"config.json"
emo_gen:config_path: '/Data/'+model_name+'/'+"config.json"
train_ms: model: '/Data/'+model_name+'/'+"models"
train_ms: config_path: '/Data/'+model_name+'/'+"config.json"
'''

with open('/content/Bert-VITS2/config.yml', 'w') as f:
  yaml.dump(config, f)

In [ ]:
# @title .json 변경 (업로드했으면 실행 X) only for 2.3
import shutil
json_where = '/content/Bert-VITS2/Data/models/config.json'
shutil.copy('/content/Bert-VITS2/configs/config.json', json_where)

import json
with open(json_where, 'r') as f:
    json_data = json.load(f)
#print(json.dumps(json_data) )
json_data['train']['batch_size'] = 6 #기본값12
json_data['train']['log_interval'] = 50 #기본값 200
#json_data['train']['eval_interval'] = 500 #기본값 1000
with open(json_where, 'w', encoding='utf-8') as make_file:
    json.dump(json_data, make_file, indent="\t")

FileNotFoundError: ignored

In [ ]:
#@markdown ### WebUI 실행
##!python webui.py

import yaml
import json
from glob import glob
import shutil
import os.path
from os import path

G_name_please = "" # @param {type:"string"}
inference_mode = "cpu" # @param ["cuda", "cpu"]
models_where = inference_arrive
latest_ckpt_name = ""

if G_name_please is "":
  latest_ckpt_G=max(glob(os.path.join(models_where, "*.pth")), key=os.path.getctime)
  #max(glob(os.path.join(models_where, "G_*.pth")), key=os.path.getctime)
  latest_ckpt_G_name = latest_ckpt_G.split('/')[-1]
else:
  latest_ckpt_G_name = G_name_please

latest_json = max(glob(os.path.join(models_where, "*.json")), key=os.path.getctime)
latest_json_name = latest_json.split('/')[-1]


with open('/content/Bert-VITS2/config.yml') as f:
  config = yaml.load(f, Loader=yaml.FullLoader)
config['dataset_path'] = "Data/"
#config['webui']['device'] = "cuda"  # cpu나 cuda. cpu로도 아무리 늦어도 1분 내로 생성. GPU로 켰다면 cuda로 변경할 것
#config['webui']['device'] = "cpu"
config['webui']['device'] = inference_mode
config['webui']['model'] = "models/" + latest_ckpt_G_name
config['webui']['share'] = "true"
#config['webui']['port'] = 7860
config['webui']['debug'] = "false"
config['webui']['config_path'] = "models/" + latest_json_name
#config['server']['models']['device'] = lllist
with open('/content/Bert-VITS2/config.yml', 'w') as f:
    yaml.dump(config, f)

'''
with open(latest_json, 'r') as f:
    json_data = json.load(f)
#json_data['train']['eval_interval'] = 500 #기본값 1000
with open(latest_json, 'w', encoding='utf-8') as make_file:
    json.dump(json_data, make_file, indent="\t")
'''

%mv webui.py web_inference.py
!python web_inference.py

mv: cannot stat 'webui.py': No such file or directory


<>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-23-2330a0330b62>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if G_name_please is "":


| numexpr.utils | INFO | NumExpr defaulting to 2 threads.
| __main__ | INFO | Enable DEBUG-LEVEL log
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
| utils | INFO | Loaded checkpoint 'Data/models/G_6000.pth' (iteration 170)
推理页面已开启!
Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://b86e2a96b2a4ef4a1a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
Building prefix dict from the default dictionary ...
| jieba | DEBUG | Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
| jieba | DEBUG | Loading model from cache /tmp/jieba.cache
Loading mo